In [4]:
import pandas as pd
import dtale
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objects as go # for visualization
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import os
import statsmodels.api as sm
import sys 
sys.path.append("../../Function/")
from function import remove_high_corr
from function import remove_outliers

%load_ext nb_black
%load_ext blackcellmagic

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


<IPython.core.display.Javascript object>

In [5]:
df = pd.read_excel("../../../site_info_ver_3.2.xlsx")



<IPython.core.display.Javascript object>

In [6]:
df = df[df.is_weekend==0]
df

,Date,Model_number,Site_Name,Israelis_Count,Tourists_Count,Total,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,operations,is_jewish_holiday,is_jewish_vacation,is_muslims_holiday,is_muslims_vacation,is_camping,special_activity,visit_duration,is_jewishHeritage,is_muslimsHeritage,is_christiansHeritage,is_nationalPark,is_natureReserve,is_archaeology,is_religion,is_animals,is_heritage,is_water,is_cave,is_lookout,pm10,pm2.5,nox,so2,Temperature,is_HeatWave,Tel_Aviv-Yafo_pm10,Jerusalem_pm10,Haifa_pm10,Ashkelon_pm10,Beer-Sheva_pm10,Tel_Aviv-Yafo_pm2.5,Jerusalem_pm2.5,Haifa_pm2.5,Ashkelon_pm2.5,Beer-Sheva_pm2.5,Tel_Aviv-Yafo_nox,Jerusalem_nox,Haifa_nox,Ashkelon_nox,Beer-Sheva_nox,Tel_Aviv-Yafo_so2,Jerusalem_so2,Haifa_so2,Ashkelon_so2,Beer-Sheva_so2,is_Site_exceeded_pm10,is_Site_exceeded_pm2.5,is_Site_exceeded_nox,is_Site_exceeded_so2,Tel_Aviv-Yafo_pm10_exceeded,Jerusalem_pm10_exceeded,Haifa_pm10_exceeded,Ashkelon_pm10_exceeded,Beer-Sheva_pm10_exceeded,Tel_Aviv-Yafo_pm2.5_exceeded,Jerusalem_pm2.5_exceeded,Haifa_pm2.5_exceeded,Ashkelon_pm2.5_exceeded,Beer-Sheva_pm2.5_exceeded,Tel_Aviv-Yafo_so2_exceeded,Jerusalem_so2_exceeded,Haifa_so2_exceeded,Ashkelon_so2_exceeded,Beer-Sheva_so2_exceeded,Tel_Aviv-Yafo_nox_exceeded,Jerusalem_nox_exceeded,Haifa_nox_exceeded,Ashkelon_nox_exceeded,Beer-Sheva_nox_exceeded,Green_border,Season_autumn,Season_spring,Season_summer,Season_winter
2,2016-01-03,5,Alexander Stream-Bet Yannai,7,0,7,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,42.9,NaN,6.3,5.5,16.6,0,182.6,339.9,83.9,33.5,42.6,66.4,86.1,69.8,19.8,15.4,478.0,621.6,49.7,4.2,39.0,10.3,0.8,6.9,1.8,3.7,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
3,2016-01-04,5,Alexander Stream-Bet Yannai,31,0,31,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,9.8,2.3,17.5,0,29.9,223.5,55.7,22.9,33.8,38.4,51.8,90.7,19.6,19.2,261.8,735.3,488.7,5.3,68.6,11.0,0.2,12.4,1.1,4.5,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
4,2016-01-05,5,Alexander Stream-Bet Yannai,33,0,33,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,85.3,66.30,22.6,2.7,18.9,0,315.5,662.0,201.0,68.5,95.8,102.3,114.8,159.8,53.9,30.5,306.0,1248.2,272.1,6.4,35.8,6.0,0.6,9.9,1.4,6.4,0,1,0,0,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
5,2016-01-06,5,Alexander Stream-Bet Yannai,15,0,15,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,NaN,14.40,18.8,6.1,19.2,0,123.9,577.5,64.6,28.2,58.6,40.8,40.1,47.2,22.6,19.8,431.3,1606.6,338.9,6.1,72.1,13.9,0.3,9.9,6.8,7.7,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
6,2016-01-07,5,Alexander Stream-Bet Yannai,23,0,23,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,NaN,18.75,9.4,2.8,20.3,0,95.5,78.5,51.1,39.7,37.2,29.1,56.3,52.2,26.4,15.6,129.6,793.6,49.7,1.1,20.3,4.1,0.1,2.0,1.4,0.3,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109099,2021-07-07,2,Baptismal Site Qasr al-Yahud,6,0,6,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,42.2,15.30,9.8,0.3,38.5,1,137.3,42.2,96.5,37.7,39.0,161.9,77.4,132.4,17.5,12.9,1255.2,709.1,156.2,4.7,22.4,4.7,2.8,8.5,1.0,6.6,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,0
109102,2021-07-11,2,Baptismal Site Qasr al-Yahud,2,0,2,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,530.3,15.90,17.1,4.0,38.5,1,270.2,530.3,29.1,44.3,67.0,254.1,81.0,89.2,17.0,21.8,2257.0,913.5,359.5,5.7,103.4,17.5,2.6,24.0,3.9,7.1,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,1,0
109104,2021-07-18,2,Baptismal Site Qasr al-Yahud,2,0,2,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,349.4,12.50,11.5,2.0,38.5,1,165.9,349.4,78.7,38.9,46.1,184.4,83.5,111.3,13.3,16.7,1371.8,1004.4,217.6,5.5,31.5,6.4,2.5,13.8,0.9,9.1,0,0,0

<IPython.core.display.Javascript object>

Check Null

In [7]:
# drops = remove_high_corr(df, target, threshold)
# df.isna().sum()
# df=df.dropna()
# df.reset_index(inplace=True,drop=True)
# df.drop(['pm10','nox','so2','pm2.5'],axis=1,inplace=True)
len(df[df.isnull().any(axis=1)])/len(df)

0.5747188623101596

<IPython.core.display.Javascript object>

Defiend X and y

In [8]:
X = df.drop(['Israelis_Count','Tourists_Count','Total',"Date","Model_number","is_weekend"],axis=1)
targets = df[['Israelis_Count','Tourists_Count','Total']]

# X_dummies = pd.get_dummies(X,'Site_Name',drop_first=True)
targets

,Israelis_Count,Tourists_Count,Total
2,7,0,7
3,31,0,31
4,33,0,33
5,15,0,15
6,23,0,23
...,...,...,...
109099,6,0,6
109102,2,0,2
109104,2,0,2
109105,10,0,10


<IPython.core.display.Javascript object>

Scaleing

In [9]:

#scale X
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X =X.drop("Site_Name",axis=1)
scaled_X = pd.DataFrame(scaler.fit_transform(scaled_X), columns=scaled_X.columns)
scaled_X["Site_Name"] = X.Site_Name

#scale y
scaled_targets = np.log(targets+0.01)
scaled_targets

scaled_df = scaled_X.merge(scaled_targets,left_index=True,right_index=True)

<IPython.core.display.Javascript object>

Model MLR with Pollutions droped Null rows

In [22]:
treshold = 0.8
Descripton = "Regular corr drop with treshold "+str(treshold)+" remove nulls rows"# and outliers" 
Model_type= "MLR"

res=pd.read_excel("../../../res.xlsx")

# targets = [scaled_targets.Israelis_Count,scaled_targets.Total,scaled_targets.Tourists_Count]
targets_list_name = ['Israelis_Count','Total','Tourists_Count']
for idx in range(3):
    tmp = scaled_df.copy()
    target_drop=[x for x in targets_list_name if x!=targets_list_name[idx]]
    tmp.drop(target_drop,axis=1,inplace=True)
    drops = remove_high_corr(tmp.drop("Site_Name",axis=1), targets_list_name[idx],treshold)
    tmp = tmp.drop(drops,axis=1)
    tmp.dropna(inplace=True)
    # tmp = remove_outliers(tmp, targets_list_name[idx])
    y = tmp[targets_list_name[idx]]
    X = tmp.drop(targets_list_name[idx],axis=1)
    #split the data, train the model and get prediction for the training and for the test  
    #######################################################################################################################
    X = pd.get_dummies(X,"",drop_first=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=312148513)
    train_df_scaled = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
    test_df_scaled = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
    
    lm = LinearRegression()  # define our model using least square method
    lm.fit(X_train,y_train)   # Fit our linear model

    #train
    fitted = np.exp(lm.predict(X_train))
    predicted_train = round(pd.Series(fitted, index=y_train.index, name='Predicted_train_'+targets_list_name[idx]),ndigits=2)
    orignal_train=np.exp(train_df_scaled[[targets_list_name[idx]]])
    train_df = pd.merge(left=orignal_train, right=predicted_train, left_index=True, right_index=True)
    train_df

    #test
    fitted = np.exp(lm.predict(X_test))
    predicted_test = round(pd.Series(fitted, index=y_test.index, name='Predicted_test_'+targets_list_name[idx]),ndigits=2)
    orignal_test = np.exp(test_df_scaled[[targets_list_name[idx]]])
    test_df = pd.merge(left=orignal_test, right=predicted_test, left_index=True, right_index=True)
    test_df
     #######################################################################################################################



     
     
     #plot the residuals graph
     #######################################################################################################################
    
     #calculate the residuals
    test_df['residuals'] = test_df['Predicted_test_'+targets_list_name[idx]] - test_df[targets_list_name[idx]]
    train_df['residuals'] = train_df['Predicted_train_'+targets_list_name[idx]] - train_df[targets_list_name[idx]]
    
    fig= go.Figure()
    fig.add_trace(
      go.Scatter(
          x=train_df['Predicted_train_'+targets_list_name[idx]],
          y=train_df.residuals,
          mode='markers',
          name='train residuals',
          marker_color='blue',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.add_trace(
      go.Scatter(
          x=test_df['Predicted_test_'+targets_list_name[idx]],
          y=test_df.residuals,
          mode='markers',
          name='test residuals',
          marker_color='red',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.add_trace(
       go.Scatter(
          x=test_df['Predicted_test_'+targets_list_name[idx]],
          y=test_df.residuals*0,
          mode='lines',
          name='zero line',
          marker_color='black',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.update_layout(
        title="Residuals of Predicted "+targets_list_name[idx],
        xaxis_title="Predicted "+targets_list_name[idx],
        yaxis_title="Residuals",
        font=dict(
            size=14,
            color="RebeccaPurple"
        )
    )
    fig.show()
     #######################################################################################################################
     
     #create folder
     ########################################
    if not os.path.exists(Descripton):
      os.mkdir(Descripton)
    fig.write_image(Descripton+"/results_"+targets_list_name[idx]+'_.png', width=1500, height=600)
    #########################################

    #create txt file with the metrix result 
    ################################################################################################################################################     
    f = open(Descripton+"/results_"+targets_list_name[idx]+'_.txt', 'w')

    #  correct_rows = train_df.loc[abs(train_df['Predicted_train_'+targets_list_name[idx]]-train_df[targets_list_name[idx]])<=50].any(axis=1).count()
    #  ACC_Training = round(correct_rows/len(train_df),3)
    if len(train_df.loc[train_df['Predicted_train_'+targets_list_name[idx]]>=np.infty])/len(train_df)*100 <1:
      train_df.drop(train_df.loc[train_df['Predicted_train_'+targets_list_name[idx]]>=np.infty].index,inplace=True)
    MSE_Training = metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    RMSE_Training = np.sqrt(metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]]))
    MAE_Training = metrics.mean_absolute_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    R2_Training=metrics.r2_score(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    std_Training = np.std(predicted_train)

    f.write("------ TRAIN DATA ------\n")
    #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Training)+"\n")
    f.write("MSE : "+str(MSE_Training)+", RMSE: "+str(RMSE_Training)+", MAE : "+str(MAE_Training)+"\n")
    f.write("R2 TRAIN "+ str(R2_Training)+"\n")


    #  correct_rows = test_df.loc[abs(test_df['Predicted_test_'+targets_list_name[idx]]-test_df[targets_list_name[idx]])<=50].any(axis=1).count()
    #  ACC_Test = round(correct_rows/len(test_df),3)
    if len(test_df.loc[test_df['Predicted_test_'+targets_list_name[idx]]>=np.infty])/len(test_df)*100 <1:
      test_df.drop(test_df.loc[test_df['Predicted_test_'+targets_list_name[idx]]>=np.infty].index,inplace=True)   
    MSE_Test = metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    RMSE_Test = np.sqrt(metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]]))
    MAE_Test = metrics.mean_absolute_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    R2_Test = metrics.r2_score(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    std_test = np.std(predicted_test)

    f.write("\n")
    f.write("------ TEST DATA ------\n")
    #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Test)+"\n")
    f.write("MSE : "+str(MSE_Test)+", RMSE: "+str(RMSE_Test)+", MAE : "+str(MAE_Test)+"\n")
    f.write("R2 TEST "+ str(R2_Test)+"\n")
    f.write("--------------------------------\n")

    f.close()
    ################################################################################################################################################     
    coeff = lm.coef_
    equation =[]
    for i in range(len(X.columns)):
         equation.append(((round(coeff[i],4))," * "+str(X.columns[i])))
      
    equation = (sorted(equation, reverse=True,key = lambda x: x[0]))
    equation = " + ".join(map(str,equation))
    equation = ""+ str(lm.intercept_)+" + "+equation
    equation= equation.replace(',', '')
    equation= equation.replace("'", "")
    equation
    #Add to excel result the new line
    ################################################################################################################################################
    Target=targets_list_name[idx]
    Site_in_this_model='ALL'
    new_row = {
      'Descripton':Descripton,
      "Target":Target,
      'Model_type':Model_type,
      # 'Model_number':Model_number,
      'Site_in_this_model':Site_in_this_model,
      "Size Train":len(train_df),
      # 'ACC_Training':ACC_Training,
      'MAE_Training':MAE_Training,
      'MSE_Training':MSE_Training,
      'RMSE_Training':RMSE_Training,
      'STD_Training':std_Training,
      'R2_Training':R2_Training,
      "Size Test":len(test_df),
      # 'ACC_Test':ACC_Test,
      'MAE_Test':MAE_Test,
      'MSE_Test':MSE_Test,
      'RMSE_Test':RMSE_Test,
      'STD_Test':std_test,
      'R2_Test':R2_Test,
      'Check':RMSE_Test<=std_test,
      'equation':equation
      }

    res=res.append(new_row,ignore_index=True)
    ################################################################################################################################################     
    
    # Index for the next target y
    idx+=1

res.to_excel("../../../res.xlsx",index=False)
res.tail(3)

,Descripton,Target,Model_type,Site_in_this_model,Size Train,MAE_Training,MSE_Training,RMSE_Training,STD_Training,R2_Training,Size Test,MAE_Test,MSE_Test,RMSE_Test,STD_Test,R2_Test,Check,equation
447,Regular corr drop with treshold 0.8 remove nul...,Israelis_Count,MLR,ALL,16545,177.768197,171317.777524,413.905518,196.429731,0.237615,7092,177.954100,181415.070008,425.928480,193.617765,0.231213,False,-4101164652305.3286 + (4245576778383.4985 * S...
448,Regular corr drop with treshold 0.8 remove nul...,Total,MLR,ALL,16545,257.749769,308607.146766,555.524209,385.741615,0.429699,7092,251.894691,303908.966354,551.279390,377.054549,0.428497,False,9758018209944.586 + (640878825938.9824 * Tel_...
449,Regular corr drop with treshold 0.8 remove nul...,Tourists_Count,MLR,ALL,16545,142.730302,158953.937926,398.690278,78.446461,0.070569,7092,139.806056,141369.165305,375.990911,76.590634,0.068079,False,-2167567651782.308 + (3518620510513.744 * Sea...


<IPython.core.display.Javascript object>

In [20]:
res[res.Target=='Israelis_Count'].tail(3)

,Descripton,Target,Model_type,Site_in_this_model,Size Train,MAE_Training,MSE_Training,RMSE_Training,STD_Training,R2_Training,Size Test,MAE_Test,MSE_Test,RMSE_Test,STD_Test,R2_Test,Check,equation
435,Regular corr drop with treshold 0.3 remove nul...,Israelis_Count,MLR,ALL,21421,180.179575,180855.012262,425.270517,214.450353,0.268521,9181,181.340247,170693.675683,413.150912,220.085791,0.286066,0.0,3.2615928967099226 + (393861977461.0738 * Tel...
438,Regular corr drop with treshold 0.3 remove nul...,Israelis_Count,MLR,ALL,21663,179.486833,185272.631515,430.433074,210.485135,0.262082,9285,180.561340,170965.083947,413.479242,215.617041,0.272358,0.0,3.139567206083938 + (17315850555263.809 * Jer...
441,Regular corr drop with treshold 0.4 remove nul...,Israelis_Count,MLR,ALL,21663,179.409223,185293.709182,430.457558,209.482416,0.261998,9285,180.791346,171032.390745,413.560625,214.345028,0.272071,False,3.007960710111788 + (1266097876472.007 * Ashk...


<IPython.core.display.Javascript object>